In [9]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# list files in MyDrive root
!ls -l /content/drive/MyDrive | sed -n '1,200p'

# if you keep datasets in a folder called 'datasets' or 'data', check those
!ls -l /content/drive/MyDrive/datasets || true
!ls -l /content/drive/MyDrive/data || true


total 963
-rw------- 1 root root 451105 Mar 14  2024 4SN23IS006.pdf
drwx------ 2 root root   4096 Jun 30  2020 Classroom
drwx------ 2 root root   4096 Sep 15 04:33 Colab Notebooks
drwx------ 2 root root   4096 Nov  7 13:15 Fak_news
-rw------- 1 root root    181 Aug 17 09:08 FINAL450.gsheet
-rw------- 1 root root  78237 Apr 18  2025 inbound7756864321535100895.pdf
drwx------ 2 root root   4096 Nov  6  2024 Manual_java
drwx------ 2 root root   4096 Apr 19  2025 MyResume
-rw------- 1 root root 434596 Oct 25  2020 WhatsApp Unknown 2020-10-03 at 11.21.49.zip
ls: cannot access '/content/drive/MyDrive/datasets': No such file or directory
ls: cannot access '/content/drive/MyDrive/data': No such file or directory


In [11]:
import pandas as pd

# Set the correct path to your files inside the Fak_news folder
fake_path = '/content/drive/MyDrive/Fak_news/Fake.csv'
true_path = '/content/drive/MyDrive/Fak_news/True.csv'

# Load the datasets
df_fake = pd.read_csv(fake_path)
df_true = pd.read_csv(true_path)

# Display first few rows to confirm data loaded correctly
print("Fake News Sample:")
display(df_fake.head())

print("True News Sample:")
display(df_true.head())


Fake News Sample:


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


True News Sample:


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [12]:
from sklearn.utils import shuffle

# Add labels to each dataset
df_fake['label'] = 'FAKE'
df_true['label'] = 'REAL'

# Combine and shuffle the datasets
df = pd.concat([df_fake, df_true], axis=0)
df = shuffle(df, random_state=42).reset_index(drop=True)

print(df.shape)
df.head()


(44898, 5)


,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",FAKE
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",REAL
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",REAL
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",FAKE
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",REAL


In [13]:
df.columns


Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [14]:
from sklearn.model_selection import train_test_split

# Features (text) and labels
X = df['text']
y = df['label']

# Split dataset: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 35918
Testing samples: 8980


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train = vectorizer.fit_transform(X_train)
tfidf_test = vectorizer.transform(X_test)


In [16]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train, y_train)


PassiveAggressiveClassifier(max_iter=50)

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(tfidf_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 99.37%

Confusion Matrix:
 [[4680   30]
 [  27 4243]]

Classification Report:
               precision    recall  f1-score   support

        FAKE       0.99      0.99      0.99      4710
        REAL       0.99      0.99      0.99      4270

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [18]:
test_news = [
    "The government announced new economic reforms today.",
    "Aliens have landed on the White House lawn, scientists confirm!"
]

new_tfidf = vectorizer.transform(test_news)
predictions = model.predict(new_tfidf)

for news, label in zip(test_news, predictions):
    print(f"\n📰 News: {news}\n🔎 Prediction: {label}")



📰 News: The government announced new economic reforms today.
🔎 Prediction: FAKE

📰 News: Aliens have landed on the White House lawn, scientists confirm!
🔎 Prediction: FAKE


In [19]:
import joblib

joblib.dump(model, '/content/fake_news_model.pkl')
joblib.dump(vectorizer, '/content/vectorizer.pkl')

print("✅ Model and vectorizer saved!")


✅ Model and vectorizer saved!
